In [567]:
"""
Initial code taken from previous lessons and prof. Squillero repo  
"""

'\nInitial code taken from previous lessons and prof. Squillero repo  \n'

In [568]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
from math import ceil

import numpy as np

In [569]:
PROBLEM_SIZE = 20
NUM_SETS = 40
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

In [570]:
def goal_check(state):
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))

def is_special(s):
    return sum(s) / PROBLEM_SIZE > 0.6

def distance(state):
    uncovered = PROBLEM_SIZE - sum(
        reduce(
            np.logical_or, 
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    for i in state.not_taken:
        if is_special(SETS[i]):
            return uncovered - 0.5
    return uncovered


In [571]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable" # sometimes an error may occur

In [572]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((0, state))
counter = 0
_, current_state = frontier.get()


In [573]:
current_state

State(taken=set(), not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})

In [574]:
SETS[0]

array([ True, False,  True, False, False, False,  True, False, False,
       False, False, False, False,  True, False, False, False,  True,
       False,  True])

In [575]:
goal_check(current_state)

False

In [576]:

# New function to determine if a set is special
def is_special(s):
    return sum(s) / PROBLEM_SIZE > 0.6

# Function to get g(n) value
def g_cost(state):
    return len(state.taken)

# Updated distance function to prioritize special sets
def distance(state):
    uncovered = PROBLEM_SIZE - sum(
        reduce(
            np.logical_or, 
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    
    # Reduce the heuristic value if the next possible set is special
    for i in state.not_taken:
        if is_special(SETS[i]):
            return uncovered - 0.5
    return uncovered

# A* approach
def astar_search():
    visited = set()
    frontier = PriorityQueue()
    start_state = State(frozenset(), frozenset(range(NUM_SETS)))
    frontier.put((0 + distance(start_state), 0, start_state))
    counter = 0

    while not frontier.empty():
        _, g_cost_current, current_state = frontier.get()
        if goal_check(current_state):
            return current_state, counter

        visited.add(current_state)

        # Prioritize special sets in the actions loop
        sorted_actions = sorted(list(current_state.not_taken), key=lambda x: -sum(SETS[x]))
        for action in sorted_actions:
            new_taken = frozenset(current_state.taken | {action})
            new_not_taken = frozenset(current_state.not_taken - {action})
            new_state = State(new_taken, new_not_taken)
            
            if new_state not in visited:
                g_new = g_cost(new_state)
                h_new = distance(new_state)
                frontier.put((g_new + h_new, g_new, new_state))
                counter += 1

    return None, counter

final_state, steps = astar_search()
print(f"Solved using A* in {steps:,} steps with {len(final_state.taken)} sets.")



Solved using A* in 117 steps with 3 sets.


In [577]:
"""
Comments on completed requests: 

1. **A* algorithm**:
   - **Heuristic H drives the algorithm**:
     - **Admissible**: The heuristic is admissible if it never overestimates the cost to reach the goal. 
     In the context of the set covering, the heuristic used (`distance`) is the count of the uncovered elements in the problem. 
     This is admissible since it always underestimates or correctly estimates the number of sets needed to cover all elements.
     - **Monotonic**: Monotonicity means the heuristic should not decrease along a path. 
     In the code, `distance` is based on the number of uncovered elements, which will not increase as more sets are taken. 
     So, it's monotonic.
   - **Build a new H**:
     - **Consider the old H (distance)**: The code uses the `distance` heuristic, which is the number of uncovered elements.
     - **What about “special sets”?**: The code introduces a function `is_special(s)` that checks if a set is "special". 
     It determines a set as special if it covers more than 60% of the total elements. 
     The `distance` heuristic function gives preference to special sets by subtracting a value (0.5) if the next possible set is special.
     - **What about the order of the sets?**: In the `astar_search()` function, 
     the actions (sets) are sorted in descending order based on their size (number of elements they cover) 
     using the line `sorted_actions = sorted(list(current_state.not_taken), key=lambda x: -sum(SETS[x]))`. 
     This ensures that larger sets (and potentially "special" sets) are prioritized.
"""

'\nComments on completed requests: \n\n1. **A* algorithm**:\n   - **Heuristic H drives the algorithm**:\n     - **Admissible**: The heuristic is admissible if it never overestimates the cost to reach the goal. \n     In the context of the set covering, the heuristic used (`distance`) is the count of the uncovered elements in the problem. \n     This is admissible since it always underestimates or correctly estimates the number of sets needed to cover all elements.\n     - **Monotonic**: Monotonicity means the heuristic should not decrease along a path. \n     In the code, `distance` is based on the number of uncovered elements, which will not increase as more sets are taken. \n     So, it\'s monotonic.\n   - **Build a new H**:\n     - **Consider the old H (distance)**: The code uses the `distance` heuristic, which is the number of uncovered elements.\n     - **What about “special sets”?**: The code introduces a function `is_special(s)` that checks if a set is "special". \n     It deter